In [1]:
from src import Tensor,cross_entropy,Adam,SGD,Linear, Module, Sequential, ReLU,dataloader
import os
import sys
import numpy as np
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

In [2]:
import sklearn.datasets
train_X, train_y = sklearn.datasets.load_digits(return_X_y=True)

In [10]:
class Net(Module):
    def __init__(self, outc):
        super().__init__()
        self.seq = Sequential(
            Linear(None, 64, bias=True),
            ReLU(),
            Linear(None, 64*2, bias=True),
            ReLU(),
            Linear(None, outc),
        )

    def forward(self, x):
        x = self.seq(x)
        return x


def train(net, optimizer, batch_size=128):
    import tqdm,math,time
    start = time.monotonic()
    for epoch in range(10):  # loop over the dataset multiple times
        losses = []
        accs = []
        
        for X, y in (bar := tqdm.tqdm(dataloader(batch_size, train_X, train_y), total=math.ceil(len(train_y)/batch_size))):

            # forward + backward + optimize
            outputs = net.forward(X)
            loss = cross_entropy(outputs, y)

            optimizer.zero_grad()
            loss.backward()
            assert loss.item() > 0, loss
            optimizer.step()

            losses.append(loss.item())
            accs.append((outputs.data.argmax(-1) == y.data).astype(float).mean())
            bar.set_description(
                f'[Epoch {epoch + 1:3}] loss: {np.mean(losses):3.3f} accuracy: {np.mean(accs)*100:3.2f}')
    return time.monotonic() - start

In [11]:
net = Net(len(np.unique(train_y)))
optimizer = SGD(net.trainable_params(), lr=0.001)
time = train(net, optimizer)
print(f'Finished Training with SGD in : {time:.4f} s')

[Epoch  10] loss: 0.909 accuracy: 72.51: 100%|██████████| 15/15 [00:00<00:00, 404.03it/s]

Finished Training with SGD in : 0.4370 s


In [13]:
net = Net(len(np.unique(train_y)))
optimizer = Adam(net.trainable_params(), lr=0.001, amsgrad=True)
time = train(net, optimizer)
print(f'Finished Training with Adam in : {time:.4f} s')

[Epoch  10] loss: 0.014 accuracy: 99.84: 100%|██████████| 15/15 [00:00<00:00, 357.61it/s]

Finished Training with Adam in : 0.4530 s
